In [1]:
import pandas as pd

df = pd.read_csv('train-data.dat', sep=',', header=None)
df.columns = ['sentence']
df['sentence'] = df['sentence'].str.replace('<.*?>', '', regex=True)  # deletes <int>
# print(df.head())
df = df[0:1000]
df.head()

vocab = pd.read_csv('vocabs.txt', sep=',', header=None)
vocab.columns = ['word', 'id']

sentences = pd.read_csv('train-data.dat', sep=',', header=None)
sentences.head()

com = []
for line in df.sentence:
    sen = ''
    for word in line.split():
      for id in vocab.id:
        if int(word) == int(id):
          sen = sen + ' ' + str(vocab.word[id]) 
    # print(sen)
    com.append(sen)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

tfidf_vectorizer = TfidfVectorizer(use_idf=True)
tfidf_vectors = tfidf_vectorizer.fit_transform(com)

tfidf = tfidf_vectors.todense()
# TFIDF of words not in the doc will be 0, so replace them with nan
tfidf[tfidf == 0] = np.nan
# Use nanmean of numpy which will ignore nan while calculating the mean
means = np.nanmean(tfidf, axis=0)
# convert it into a dictionary for later lookup
means = dict(zip(tfidf_vectorizer.get_feature_names(), means.tolist()[0]))

tfidf = tfidf_vectors.todense()
# Argsort the full TFIDF dense vector
ordered = np.argsort(tfidf*-1)
words = tfidf_vectorizer.get_feature_names()

# top_k = 5
# for i, doc in enumerate(com):
#     result = {}
#     # Pick top_k from each argsorted matrix for each doc
#     for t in range(top_k):
#         # Pick the top k word, find its average tfidf from the
#         # precomputed dictionary using nanmean and save it to later use
#         result[words[ordered[i,t]]] = means[words[ordered[i,t]]]
#     # print (result)

# means

# from collections import Counter 

# means = dict(Counter(means).most_common(1000))


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [25]:
li = [[]]

for key in means:
  li.append([key, means[key]])

li.pop(0)
li[:5]

df2 = pd.DataFrame(li)
# df2 = df2.transpose()
df2.head()

tfidf_sc = df2[1]
print(tfidf_sc)

0       0.180064
1       0.105520
2       0.098096
3       0.163385
4       0.138688
          ...   
6848    0.110620
6849    0.151121
6850    0.124428
6851    0.184492
6852    0.223934
Name: 1, Length: 6853, dtype: float64


In [ ]:
import random

import matplotlib.pyplot as plt
from deap import base
from deap import creator
from deap import tools

POP_SIZE = 20
P_CROSSOVER = 0.6
P_MUTATION = 0.01
MAX_GENERATIONS = 200
GENERATIONS_EXIT = 20  # state the number of generations in which maxFitness remains the same,  for stopping condition

toolbox = base.Toolbox()

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [ ]:
from functools import partial


def fill_with_random(tfidf_sc):
    tfidf_sc = np.copy(tfidf_sc)
    nan_number = np.isnan(tfidf_sc).sum()
    filler = np.random.randint(1, 6, nan_number, dtype='int')
    mask = np.isnan(tfidf_sc)
    tfidf_sc[mask] = filler
    # return user.astype(int) for OX, PMX crossover operators
    return tfidf_sc


individual_gen = partial(fill_with_random, tfidf_sc)

functools.partial(<function fill_with_random at 0x7fd8082cf170>, 0       0.180064
1       0.105520
2       0.098096
3       0.163385
4       0.138688
          ...   
6848    0.110620
6849    0.151121
6850    0.124428
6851    0.184492
6852    0.223934
Name: 1, Length: 6853, dtype: float64)


In [ ]:
toolbox.register('attr_bool', np.random.rand)
toolbox.attr_bool()

toolbox.register('individual', lambda individual, individual_gen: individual(individual_gen().tolist()),
                     creator.Individual,
                     individual_gen)
toolbox.register("populationCreator", tools.initRepeat,
                     list, toolbox.individualCreator)
toolbox.individual()

[0.1800637157455013,
 0.10552039112682099,
 0.0980961326839355,
 0.16338486756798426,
 0.13868836996642958,
 0.06320851087684597,
 0.049556719880602874,
 0.3689221884030387,
 0.08765374120153646,
 0.08086225346534207,
 0.06341901016708112,
 0.06449886347540754,
 0.08684271439086304,
 0.18160701979288793,
 0.05250809222821281,
 0.1122891221925232,
 0.0705466567699844,
 0.06709059485705816,
 0.14239744770089752,
 0.08019538872020565,
 0.09331748181964872,
 0.09472504869744018,
 0.08503483892343919,
 0.0944280854389867,
 0.13029738724525045,
 0.2870620276752256,
 0.09284623716124929,
 0.21226363755415367,
 0.15385514338661593,
 0.06200295036749787,
 0.08426292831556093,
 0.17885175011273532,
 0.11348790050804144,
 0.08342588790735865,
 0.07673884956116038,
 0.0681147229344553,
 0.06666487041059756,
 0.0602447147181049,
 0.06819934627206747,
 0.19550402659576716,
 0.08866878792453084,
 0.40752174759480003,
 0.10013327808090512,
 0.07763601718822008,
 0.13202851340462918,
 0.083432404657830